In [1]:
#!/usr/bin/env python3
# coding=utf-8
"""A simple example demonstrating how to use Argparse to support subcommands.
This example shows an easy way for a single command to have many subcommands, each of which takes different arguments
and provides separate contextual help.
"""
import argparse
import cmd2
import newspaper as np
from dateutil.parser import parse
from roundup_db1 import Entry, Category, Keyword, Publication, Author, Section, DataAccessLayer
import app,roundup_help
import sys, glob, datetime, os
import BTCInput2 as btc
import itertools as it
import warnings, functools, pprint

In [2]:
# create the top-level parser for the base command
base_parser = argparse.ArgumentParser()
base_subparsers = base_parser.add_subparsers(title='subcommands', help='subcommand help')

# create the top-level parser for the search command
search_parser = argparse.ArgumentParser()
search_subparsers = search_parser.add_subparsers(title='subcommands', help='subcommand help')

# create the parser for the "entry" subcommand
parser_entry = search_subparsers.add_parser('entry', help='entry help')
parser_entry.add_argument('-id', '--entry_id', help='id of the entry')
parser_entry.add_argument('-idr', '--id_range', nargs=2,
                              help='minimum and maximum ids', )
parser_entry.add_argument('-d', '--date', help='date of entry')
parser_entry.add_argument('-l', '--url', help='article_url')
parser_entry.add_argument('-t', '--title', nargs='*', help='entry name')
parser_entry.add_argument('-dr', '--date_range', nargs=2, help='start date, end date')
parser_entry.add_argument('-c', '--category_id', help='category id')
parser_entry.add_argument('-desc',  '--description', nargs='*',
                           help='article description (optional)')

#create the parser for the "category" subcommand
parser_category = search_subparsers.add_parser('category', help='category help')
parser_category.add_argument('-id', '--category_id', help='category id')
parser_category.add_argument('-n', '--category_name', nargs='*', help='category name e.g. Python')
parser_category.add_argument('-s', '--section_id', help='section id')


#create the parser for the "section" subcommand
parser_section = search_subparsers.add_parser('section', help='section help')
parser_section.add_argument('-id', '--section_id', help='section id')


class RoundupCMD(cmd2.Cmd):
    """
    Example cmd2 application where we a base command which has a couple subcommands
    and the "sport" subcommand has tab completion enabled.
    """
    def __init__(self):
        super().__init__()

    def base_category(self, args):
        '''Generic category search function'''
        app.find_category(session=a.d.session, args=args)
        
    
    def base_entry(self, args):
        '''Generic entry search function'''
        app.find_entry(session=a.d.session, args=args)
    
    def base_section(self, args):
        '''Generic section search function'''
        pass
        
    parser_category.set_defaults(func=base_category)
    parser_entry.set_defaults(func=base_entry)

    @cmd2.with_argparser(search_parser)
    def do_search(self, args):
        """Search command help"""
        func = getattr(args, 'func', None)
        if func is not None:
            # Call whatever subcommand function was selected
            func(self, args)
        else:
            # No subcommand was provided, so call help
            self.do_help('search')
            
    #create universal add entry functionality
            
    finalize_parser = argparse.ArgumentParser()
    finalize_parser.add_argument('-d', '--date', help='search a single date')
    finalize_parser.add_argument('-r', '--date_range', nargs=2,
                                 help='search the dates between the start and end dates')
    
    @cmd2.with_argparser(finalize_parser)
    def do_finalize(self, args):
        '''searches for articles without descriptions and lets user edit them'''
        if args.date:
            app.finalize2(session=a.d.session, start_date =args.date, end_date=args.date)
        elif args.date_range:
            app.finalize2(session=a.d.session, start_date = args.date_range[0],
                          end_date=args.date_range[1])
        else:
            print('Please enter date or date range. Check help for details')
            return
        
    def do_export_docx(self, line):
        '''Exports a roundup in docx form. The "line" argument is deleted by the function'''
        app.create_docx_roundup(line)
    
    export_parser = argparse.ArgumentParser()
    export_parser.add_argument('-t', '--title', nargs='*', help='roundup title')
    export_parser.add_argument('-f',
        '--filename', help='filename (same directory as the app)')
    export_parser.add_argument('-r', '--date_range', nargs=2,
                                 help='search the dates between the start and end dates')
    
    @cmd2.with_argparser(export_parser)
    def do_export_html(self, args):
        '''Export an html version of the roundup'''
        warnings.warn('export_html implementation in testing phase')
        app.export_html2(session=a.d.session, program=args.filename,
                        start_date=parse(args.date_range[0]).date(),
                         end_date=parse(args.date_range[1]).date(),
                        title=' '.join(args.title))
    
    def do_exit(self, arg):
        '''Exits the program, any existing database connections will be closed'''
        a.close()
        print('Exiting Roundup Generator')
        return True

In [3]:
if __name__ == '__main__':
    #import sys
    #app = SubcommandsExample()
    #sys.exit(app.cmdloop())
    a=app.App()
    a.setup()
    RoundupCMD().cmdloop()

-f is not a recognized command, alias, or macro
/Users/thomassullivan/Library/Jupyter/runtime/kernel-87f3d5a9-8ac9-4801-83ac-a0398142e062.json is not a recognized command, alias, or macro


 search category -n scr


['scr']
scr
1 categories found


1 to view results, 2 to cancel:  1
1 to view next, 2 to quit:  1


scripting/automation
Category(id='2' name='scripting/automation', section=1)


1 to view next, 2 to quit:  2


scripting/automation
returning to main menu


 exit


Exiting Roundup Generator


In [10]:
a.d.session.close()